In [ ]:
"""
Detection Classifier Generation
"""

import os
import numpy as np
import torch
import easydict 
import random
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as dset
from torch.utils.data import DataLoader
from torchvision import transforms
from utils.deepsvdd import DeepSVDD_network, pretrain_autoencoder, TrainerDeepSVDD
from threshold_optimizer import ThresholdOptimizer


In [ ]:
# Specify the cuda setting if you needed
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [ ]:
# IMPORTANT
# PLEASE CHANGE THE VAULUE OF DeepTaster_forder TO THE DEEPTASTER DIRECTORY 
Home_directory = '/data/seonhye/sorce'
DeepTaster_forder = Home_directory+"/DeepTaster/"

In [ ]:
# load train adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/train',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

train_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=True,
                                        num_workers=8)

In [ ]:
# load test adversarialDFT dataset
dataset = dset.ImageFolder(root=DeepTaster_forder +'images/val',
                            transform=transforms.Compose([
                                transforms.Grayscale(),
                                transforms.Resize(28),      
                                transforms.CenterCrop(28), 
                                transforms.ToTensor(),    
                            ]))

val_dataloader = torch.utils.data.DataLoader(dataset,
                                        batch_size=16,
                                        shuffle=False,
                                        num_workers=8)

In [ ]:
# Specify the cuda setting if you needed
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = easydict.EasyDict({
        'num_epochs':3,
        'num_epochs_ae':40,
        'lr':1e-3,
        'lr_ae':1e-3,
        'weight_decay':5e-7,
        'weight_decay_ae':5e-3,
        'lr_milestones':[50],
        'batch_size':1024,
        'pretrain':True,
        'latent_dim':32,
        'normal_class':0
                })

In [ ]:
deep_SVDD = TrainerDeepSVDD(args, train_dataloader,device, DeepTaster_forder +'models/classifier/pretrained.pth')
if args.pretrain:
    deep_SVDD.pretrain()
net = DeepSVDD_network().to(device)
state_dict = torch.load(DeepTaster_forder +'models/classifier/pretrained.pth')
net.load_state_dict(state_dict['net_dict'])
c = torch.Tensor(state_dict['center']).to(device)


In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=args.lr,
                        weight_decay=args.weight_decay)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, 
            milestones=args.lr_milestones, gamma=0.1)


In [ ]:
net.train()
for epoch in range(args.num_epochs):
    total_loss = 0
    for x, _ in train_dataloader:
        x = x.float().to(device)

        optimizer.zero_grad()
        z = net(x)
        loss = torch.mean(torch.sum((z - c) ** 2, dim=1))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    scheduler.step()
    print('Training Deep SVDD... Epoch: {}, Loss: {:.3f}'.format(
            epoch+1, total_loss/len(train_dataloader)))


In [ ]:
scores = []
label = []
net.eval()
with torch.no_grad():
    for x, y in val_dataloader:
        x = x.float().to(device)
        z = net(x)
        score = torch.sum((z - c) ** 2, dim=1).cpu()
        for i in range(16):
            if y[i]==0:
                scores.append(score[i])
                label.append(0)
            else:
                scores.append(score[i])
                label.append(1)      

In [ ]:
scores.sort()

In [ ]:
threshold = float(scores[862])

In [ ]:
print("threshold: ", threshold)
threshold_file=open(DeepTaster_forder +'models/classifier/threshold.txt','w')
threshold_file.write(str(threshold))
threshold_file.close()
torch.save(net, DeepTaster_forder +'models/classifier/deepsvdd.pt') 